In [1]:
from imutils import face_utils
# from imutils.helpers import FACIAL_LANDMARKS_IDXS
# from imutils.helpers import shape_to_np
from imutils.face_utils import rect_to_bb
import argparse
import imutils
import dlib
import numpy as np
import cv2

In [2]:
def scaleImage(predictor, image, gray, rect) :
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    leftEyePts = shape[lStart:lEnd]
    rightEyePts = shape[rStart:rEnd]
    leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
    rightEyeCenter = rightEyePts.mean(axis=0).astype("int")
    print(leftEyeCenter, rightEyeCenter)
    dY = rightEyeCenter[1] - leftEyeCenter[1]
    dX = rightEyeCenter[0] - leftEyeCenter[0]
    angle = np.degrees(np.arctan2(dY, dX)) - 180
    dist = np.sqrt((dX ** 2) + (dY ** 2))
    desiredDist = 128
    scale = desiredDist / dist
    eyesCenter = ((leftEyeCenter[0] + rightEyeCenter[0]) // 2,
        (leftEyeCenter[1] + rightEyeCenter[1]) // 2)
    M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)
    (w, h) = image.shape[:2]
    output = cv2.warpAffine(image, M, (w, h),
        flags=cv2.INTER_CUBIC)
    return output

In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [4]:
image = cv2.imread('face.jpeg')
image = imutils.resize(image, width=800)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow("Input", image)
if cv2.waitKey(1) & 0xFF == ord('q'):
    cv2.destroyAllWindows()
rects = detector(gray, 2)

In [ ]:
for rect in rects:
    (x, y, w, h) = rect_to_bb(rect)
    faceOrig = imutils.resize(image[y:y + h, x:x + w], width=256)
    faceAligned = scaleImage(predictor,image,gray,rect)
    cv2.imshow("Original", faceOrig)
    cv2.imshow("Aligned", faceAligned)
    cv2.waitKey(0)

[567 506] [261 511]
